In [1]:
import pandas as pd
import os 
from scipy.interpolate import interp1d
from utils.data_plotting import *

### Load the texture data into a df

In [2]:
texture_df = pd.read_csv("texturas_extraidas_cast.csv")

### Create bins of 1cm (calling it TDEP)

In [3]:
# The interval we want
step = 0.01

# Floor division and round to the nearest below
texture_df['TDEP'] = (texture_df['Profundidade'] // step) * step

### Round all the texture variables to 4

In [4]:
texture_df = texture_df.round(4)

In [5]:
texture_df.to_csv('texturas_bins.csv', index=False)

### Use the mean and group by the bins

In [6]:
stats = {
    'contrast':      'mean',
    'dissimilarity': 'mean',
    'homogeneity':   'mean',
    'energy':        'mean',
    'correlation':   'mean',
    'entropy':       'mean'
}

results = texture_df.groupby('TDEP').agg(stats).reset_index()

### Round TDEP (e.g. 467.84000000000003 to 467.84)

In [7]:
results['TDEP'] = results['TDEP'].round(2)
results = results.round(4)

### Save the data into a CSV

In [8]:
results.to_csv('texturas_cast_res_preprocessed.csv', index=False)

### Get the convetional well logs from the same well

In [9]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed_cast', '3-BRSA-778-SE', 'logical_file_0', 'frame_0.csv')
conventional_data = pd.read_csv(base_path)

### Testing with GR, RHOB and DRHO first

In [10]:
conventional_data = conventional_data.iloc[::-1].reset_index(drop=True)

gr_data = conventional_data[['TDEP', 'GR']]
gr_data = gr_data.dropna(subset=['GR'])

rhob_drho_data = conventional_data[['TDEP', 'RHOB', 'DRHO']]
rhob_drho_data = rhob_drho_data.dropna(subset=['RHOB', 'DRHO'])

### Interpolate conventional well logs to 1cm res (Trying with GR first)

In [11]:
def interpolate_conv_logs(df, well_log):
    # 1. Crie o interpolador a partir dos seus dados originais de GR
    interp_func = interp1d(
        df['TDEP'],        # prof. originais do gr
        df[well_log],          # valores de GR originais
        kind='linear',          # cria uma linha reta entre dois vals de GR.
    )

    # 2. Usando o intervalo de profundidade da imagem CAST
    nova_tdep = results['TDEP'].values

    min_gr_tdep = df['TDEP'].min()
    max_gr_tdep = df['TDEP'].max()
    nova_tdep = nova_tdep[(nova_tdep >= min_gr_tdep) & (nova_tdep <= max_gr_tdep)]

    # 3. Gera valores de GR interpolados seguindo a reta entre dois pontos
    interp1d_well_log = interp_func(nova_tdep)

    # 4. criando o df
    df_interp1d = pd.DataFrame({
        'TDEP': nova_tdep,
        well_log: interp1d_well_log
    })

    # 5. Arredonda os valores de GR pra 4 casas
    df_interp1d[well_log] = df_interp1d[well_log].round(4)

    return df_interp1d

In [12]:
gr_alinhado = interpolate_conv_logs(gr_data, 'GR')
rhob_alinhado = interpolate_conv_logs(rhob_drho_data, 'RHOB')
drho_alinhado = interpolate_conv_logs(rhob_drho_data, 'DRHO')

### Remove data where we did not have CAST logs

In [13]:
def remove_without_cast(df, df_interp1d):
    min_tdep = df_interp1d['TDEP'].min()
    max_tdep = df_interp1d['TDEP'].max()

    df = df[(df['TDEP'] >= min_tdep) & (df['TDEP'] <= max_tdep)]
    df = df.reset_index(drop=True)
    return df

In [14]:
gr_data = remove_without_cast(gr_data, gr_alinhado)
rhob_drho_data = remove_without_cast(rhob_drho_data, rhob_alinhado)

### Plotting original conventional and interpolated well logs

In [15]:
from matplotlib.backends.backend_pdf import PdfPages

# plot_gr_logs(gr_fino, "GR - High Resolution")

plot_gr_logs_2_wells_high_res(gr_data, gr_alinhado, "GR - Normal X High Resolution")
# Nome do arquivo PDF onde o gráfico será salvo
pdf_filename = "gr_normal_high_resolution_04.pdf"

# Criar o PDF e salvar o gráfico atual
with PdfPages(pdf_filename) as pdf:
    pdf.savefig()  # Salva o gráfico atual
    plt.close()    # Fecha o gráfico para evitar sobreposição

print(f"Gráfico salvo em {pdf_filename}")

Gráfico salvo em gr_normal_high_resolution_04.pdf


In [16]:
from matplotlib.backends.backend_pdf import PdfPages

# plot_gr_logs(gr_fino, "GR - High Resolution")
plot_rhob_drho_high_res(
    rhob_drho_data[['TDEP', 'RHOB']], 
    rhob_alinhado, 
    rhob_drho_data[['TDEP', 'DRHO']],
    drho_alinhado,
    "Interpolacao RHOB",
    "Interpolacao DRHO"
)

# Nome do arquivo PDF onde o gráfico será salvo
pdf_filename = "rhob_drho_v01.pdf"

# Criar o PDF e salvar o gráfico atual
with PdfPages(pdf_filename) as pdf:
    pdf.savefig()  # Salva o gráfico atual
    plt.close()    # Fecha o gráfico para evitar sobreposição

print(f"Gráfico salvo em {pdf_filename}")

Gráfico salvo em rhob_drho_v01.pdf


### Merging the dataframes with the conventional well logs and the CAST logs

In [17]:
merged_gr = pd.merge(
    results,
    gr_alinhado, 
    on='TDEP',
    how='left'
)

merged_rhob = pd.merge(
    merged_gr,
    rhob_alinhado, 
    on='TDEP',
    how='left'
)

merged_drho = pd.merge(
    merged_rhob,
    drho_alinhado, 
    on='TDEP',
    how='left'
)

print(merged_drho.head())

merged_drho.to_csv('cast_gr_rhob_drho.csv', index=False)

     TDEP  contrast  dissimilarity  homogeneity  energy  correlation  entropy  \
0  427.12       0.0            0.0          1.0     1.0          1.0      0.0   
1  427.13       0.0            0.0          1.0     1.0          1.0      0.0   
2  427.14       0.0            0.0          1.0     1.0          1.0      0.0   
3  427.15       0.0            0.0          1.0     1.0          1.0      0.0   
4  427.16       0.0            0.0          1.0     1.0          1.0      0.0   

        GR    RHOB    DRHO  
0  97.0781  2.1704 -0.4605  
1  97.3229  2.1703 -0.4603  
2  97.5676  2.1701 -0.4601  
3  97.8123  2.1700 -0.4600  
4  98.0570  2.1699 -0.4598  
